# **Extract Transform Load (ETL) Lab**


## **Objectives**
Write a simple ETL program
*   Read CSV and JSON file types.
*   Extract data from CSV and JSON file types.
*   Transform data.
*   Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.


### Import the required modules and functions


In [ ]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

### Download Files


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

## Unzip Files


In [ ]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


## About the Data


The file `dealership_data` contains CSV, JSON, and XML files for used car data which contain features named `car_model`, `year_of_manufacture`, `price`, and `fuel`.


## Set Paths


In [ ]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

# **Extract Phase**


### CSV Extract Function

In [ ]:
# Add the CSV extract function below
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

### JSON Extract Function


In [ ]:
# Add the JSON extract function 
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

### XML Extract Function


In [ ]:
# Add the XML extract function 
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model', 'year_of_manufacture', 'price',  'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = float(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe

###  Extract Function

Call the specific extract created functions above.

In [ ]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

<details><summary>Click here for the solution</summary>

```
    
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data
```

</details>


# **Transform Phase**


In [ ]:
# Round the price columns to 2 decimal places
def transform(data):
        data['price'] = round(data.price,2)
        return data

# **Loading Phase**


In [ ]:
# Add the load function
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

## **Logging**


*   Change the name of the logfile to the one specified in the set paths section. 
*   Change the timestamp order to Hour-Minute-Second-Monthname-Day-Year.




In [ ]:
# Add the log function 
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' # Hour-Minute-Second-Monthname-Day-Year.
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## **Running ETL Process**


###  ETL Process

*   Run all functions to extract, transform, and load the data. 
*   logging all events using the `log` function.


In [ ]:
# Log that the ETL process has started 
log("ETL Job Started")

# Log that the Extract step has started 
# Call the Extract function
# Log that the Extract step have completed

log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")


# Log that the Transform step has started
# Call the Transform function
# Log that the Transform step has completed 

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")


# Log that the Load step has started
# Call the Load function
# Log that the Load step has completed
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

# Log that the ETL process has completed!
log("ETL Job Ended")